In [1]:
import json
import pandas as pd

In [2]:
import plotly.express as px
import plotly.graph_objects as go

import dash
from dash import html, dcc
from dash.dependencies import Input, Output

### Load the data

In [3]:
df_upcstoreweek = pd.read_csv('../data/relrevenue_per_storeupcweek.csv')
df_upcstore = pd.read_csv('../data/relrevenue_per_upcstore.csv')
df_upc = pd.read_csv('../data/relrevenue_per_upc.csv')

In [4]:
df_upc.head()

,Unnamed: 0,upc,marketing,relative_revenue
0,0,3800039118,N,1.000000
1,1,1111087396,N,1.000000
2,2,31254742835,F,1.876145
3,3,7110410471,D,1.842453
4,4,3000006560,F,2.304283


### Plot Funktionen (mit test)

In [5]:
def plot_allupc():
    fig = px.strip(
        df_upc.set_index('marketing').loc[['N', 'T', 'D', 'F', 'FD']].reset_index(), 
        y = 'relative_revenue', 
        x = 'marketing',
        #color = 'upc',
        custom_data = ['upc'],
        title = 'Effect per UPC'
    )
    return fig
plot_allupc()

In [6]:
def plot_upcstore(upc):
    fig = px.strip(
        df_upcstore[df_upcstore.upc == upc], 
        y = 'relative_revenue', 
        x = 'marketing',
        #color = 'store_id',
        category_orders = {'marketing': ['N', 'T', 'D', 'F', 'FD']},
        title = f'Effect for upc {upc} per Store',
        custom_data = ['upc', 'store_id']
    )
    return fig
    
plot_upcstore(3800039118)

In [7]:
def plot_upcstoreweek(upc, store_id):
    fig = px.strip(
        df_upcstoreweek[(df_upcstoreweek.UPC == upc) & (df_upcstoreweek.store_id == store_id)], 
        y = 'relative_revenue', 
        x = 'date', 
        color = 'marketing',
        title = f'Weekly UPC: {upc} @ {store_id}'
    )
    return fig
plot_upcstoreweek(3800039118, 367)

In [8]:
def create_default_figure():
    # You can customize this as per your need. Here's an example:
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[], y=[])) # Empty Scatter Plot
    fig.update_layout(
        title="No data selected",
        xaxis_title="X-axis",
        yaxis_title="Y-axis"
    )
    return fig

create_default_figure()

## Create Dashboard

In [9]:
app = dash.Dash(__name__)
app.layout = html.Div([
    html.H1("Marketing Effects (relative Revenue)"),

    # Komponente zum Anzeigen der Callbackdaten
    html.Div([
        dcc.Markdown("""**Click Data**"""),
        html.Pre(id='jsonVis'), # Pre-formatted text element
    ]),
    
    # Hauptgrafik
    dcc.Graph(
        id='main_graph',
        figure=plot_allupc()
    ),
    
    # Zoomstufe 1
    dcc.Graph(
        id='storeupc_graph'
    ),

    # Komponente zum Anzeigen der Callbackdaten
    html.Div([
        dcc.Markdown("""**Click Data**"""),
        html.Pre(id='jsonVis2'), # Pre-formatted text element
    ]),
    
    # Zoomstufe 2
    dcc.Graph(
        id='storeupcweek_graph'
    ),
])

# ===================================
# Callbacks
# ===================================

@app.callback(
    Output(component_id='jsonVis', component_property='children'), # updates the text in the jsonVis component 
    Input(component_id='main_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
)
def display_click_data(clickData): #  It displays the click data in JSON format.
    return json.dumps(clickData, indent=2)


@app.callback(
    Output(component_id='jsonVis2', component_property='children'), # updates the text in the jsonVis component 
    Input(component_id='storeupc_graph', component_property='clickData') # whenever a point on the main_graph is clicked.
)
def display_click_data(clickData): #  It displays the click data in JSON format.
    return json.dumps(clickData, indent=2)

@app.callback(
    Output(component_id='storeupc_graph', component_property='figure'), # updates the figure of the storeupc_graph
    Input(component_id='main_graph', component_property='clickData') # whenever there is a click on a main_graph element
)
def update_fig2(clickData):
    if clickData:
        upc = clickData['points'][0]['customdata'][0]
        return plot_upcstore(upc)
    return create_default_figure()

@app.callback(
    Output(component_id='storeupcweek_graph', component_property='figure'),
    Input(component_id='storeupc_graph', component_property='clickData'))
def update_fig3(clickData):
    if clickData:
        upc = clickData['points'][0]['customdata'][0]
        store_id = clickData['points'][0]['customdata'][1]
        return plot_upcstoreweek(upc, store_id)
    return create_default_figure()

Open Dashboard at `plot.{container_id}.laboration.hochschule-stralsund.de`
You find the container_id in the address of you browser

In [10]:
app.run(jupyter_mode="tab", debug=True, host='127.0.0.1', port='8081')

Dash app running on http://127.0.0.1:8081/


<IPython.core.display.Javascript object>